In [32]:
# Or you can use a .env file
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)  # .env is in the current Module_1 folder

True

In [33]:
import uuid
thread_id = uuid.uuid4()

In [34]:
from langsmith import traceable
from groq import Groq
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever
import os
import random

groq_key = os.getenv("GROQ_API_KEY")
groq_client = Groq(api_key=groq_key)
nest_asyncio.apply()
retriever = get_vector_db_retriever()

# Fun: Add a random emoji to every system prompt for a more engaging assistant!
def get_random_emoji():
    emojis = ["🤖", "💡", "📚", "✨", "🧠", "🔍", "📝", "🚀"]
    return random.choice(emojis)

@traceable(run_type="chain")
def retrieve_documents(question: str):
    docs = retriever.invoke(question)
    print(f"[Retriever] Retrieved {len(docs)} documents for: '{question}'")
    return docs

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = f"""{get_random_emoji()} You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the latest question in the conversation.\nIf you don't know the answer, just say that you don't know.\nUse three sentences maximum and keep the answer concise."""
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    print(f"[System Prompt] {rag_system_prompt}")
    return call_groq(messages)

@traceable(run_type="llm")
def call_groq(
    messages: List[dict], model: str = "moonshotai/kimi-k2-instruct-0905", temperature: float = 0.0
) -> str:
    print(f"[LLM] Calling model: {model} with {len(messages)} messages.")
    return groq_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable(run_type="chain")
def langsmith_rag(question: str):
    print(f"[RAG] Processing question: {question}")
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    print(f"[RAG] Response received.")
    return response.choices[0].message.content


In [35]:
question = "What are three creative ways to visualize data in Python?"
ai_answer = langsmith_rag(question)
print(f"\U0001F4C8 Creative Answer: {ai_answer}")

[RAG] Processing question: What are three creative ways to visualize data in Python?
[Retriever] Retrieved 4 documents for: 'What are three creative ways to visualize data in Python?'
[System Prompt] 🚀 You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
[LLM] Calling model: moonshotai/kimi-k2-instruct-0905 with 2 messages.
[RAG] Response received.
📈 Creative Answer: 1. **Plot as Interactive Network**: Use `networkx` plus PyVis or Plotly to turn your refund graph into a responsive, drag-n-drop graph—hovering on nodes or edges in a Jupyter/IPython environment instantly reveals metadata without extra code.  
2. **Sankey + State Steps**: Transform every step (`gather_info → lookup → END`, refund→END) into a Sankey diagram (`plotly Sankey`) whose colored ribbons show where users fl

In [36]:
question = "Suggest a fun project that combines AI and art."
ai_answer = langsmith_rag(question)
print(f"\U0001F3A8 AI+Art Idea: {ai_answer}")

[RAG] Processing question: Suggest a fun project that combines AI and art.
[Retriever] Retrieved 4 documents for: 'Suggest a fun project that combines AI and art.'
[System Prompt] 🤖 You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
[LLM] Calling model: moonshotai/kimi-k2-instruct-0905 with 2 messages.
[RAG] Response received.
🎨 AI+Art Idea: Build a "museum tour" generative chatbot that asks visitors for three words and instantly produces an imaginary artwork’s image (DALL·E or SD) plus a back-story, then grades itself by simulating how long the visitor keeps talking about the piece via multi-turn conversation simulation with `langsmith + openevals`.
[RAG] Response received.
🎨 AI+Art Idea: Build a "museum tour" generative chatbot that asks visitors for three words and instant